In [ ]:
!pip install bert-for-tf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 182 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=e99d95c102694e9ba1a79a3f25d6e1994aaf5762cd38818b198d07383795aa0d
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=cf00e5899af528fa80895536da406abc316382fe7b27f15c50a155a53963b3f8
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=5ebcc4f598dc528b0ed2654dbd968ee6ef8807679135ea9e65ef9c0dda9e6bf9
  Stored in directory: /root/.cache/pip/wheels/ac/26/e9/df16869ccbd4abf517f1ff3be9a2c7ee5c5980fc87eea04fb1
Successfully built bert-for-tf2 params-flow p

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 27.6 MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!git clone https://github.com/tkeldenich/BERT_Easy_Implementation &> /dev/null

In [ ]:
import pandas as pd

In [ ]:
MT_Covid= pd.read_csv ('/content/sample_data/MT_Covid (1).csv', encoding='ISO-8859-1')

In [ ]:
MT_Covid.shape

(32, 1)

In [ ]:
MT_Covid.head(32)

id,text,Sentiment
0   254,"Art taoïste, l'acupuncture est connue pou...
1   255,L'Organisation Mondiale de la Santé (OMS) ...
2   256,"Sur l'importance de la médecine tradition...
3   257,"En Afrique et en Asie, 80 pour cent de la...
4   258," L'utilisation des plantes médicinales es...
5   259,"Il ne faut jamais acheter de produits à v...
6   260,"La médecine traditionnelle et l'éducation...
7   261,Positionnement du patient au cur de la ré...
8   262,"Plusieurs plantes ont été identifiées par...
9   263,"Selon l'Anses, ces plantes sont susceptib...
10  264,"Plébiscitée par les adeptes des solutions...
11  265," ""les conclusions (de l'alerte de l'Anse...
12  266,"La médecine traditionnelle, ont-ils dit, ...
13  267,L'Organisation Mondiale de la Santé (OMS) ...
14  268,"La Grèce d'Hippocrate a mis au point un s...
15  269,"""A n'en pas douter, le circuit officinal...
16  270,"La population d'origine africaine de Colo...
17  271,Quelles sont les plantes qui ne sont pas b...
18  272,Les plantes à éviter au jardin sont tres b...
19  273,Datura (Datura stramonium) La datura (Datu...
20  274,Colchique (Colchicum autumnale) ...est tre...
21  275,Digitale pourpre (Digitalis purpurea) ...s...
22  276,"L'Anses recommande à toutes les personnes...
23  277,Muguet (Convallaria majalis) plus intressa...
24  278,"Les études cliniques menées selon certain...
25  279,"La population dascendance africaine dépl...
26  280,"Des plantes médicinales telles que l'arte...
27  281,"La lutte contre la pandémie de Covid -19 ...
28  282,"La médecine traditionnelle précède la méd...
29  283," Plante digestive par excellence, la saug...
30  284,"D'autres plantes comme le Ginkgo biloba, ...
31  285,"Dans le but d'étudier l'aspect toxicologi...

In [ ]:
import re #

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
print(MT_Covid.columns.values)

['id' 'text' 'Sentiment']


In [ ]:
MT_Covid.Sentiment.unique()

array([1, 0])

In [ ]:
reviews= MT_Covid['text']
y = MT_Covid['Sentiment']

In [ ]:
print(reviews[10])

Plébiscitée par les adeptes des solutions naturelles, la médecine douce apporte du bien-être via une large gamme de pratiques qui régulent les énergies, font baisser l'anxiété, améliorent les sensations de douleur et de fatigue sans passer par la case médicaments.


In [ ]:
print(y[10])

1


In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
#tester la fonction tokenize
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("dont be so judgmental"))

[2123, 2102, 2022, 2061, 8689, 2389]

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [ ]:
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))


In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 219), dtype=int32, numpy=
 array([[14757, 23361,  1006, ...,     0,     0,     0],
        [10861, 20434,  2365, ...,     0,     0,     0],
        [ 3269,  2063, 17886, ...,     0,     0,     0],
        ...,
        [ 2474,  2313,  1040, ...,     0,     0,     0],
        [ 2474,  2313,  1040, ...,     0,     0,     0],
        [ 1048,  1005,  2019, ...,  1037, 10241,  1012]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 0], dtype=int32)>)

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
1/1 [==============================] - 0s 340ms/step - loss: 0.3004 - accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 290ms/step - loss: 0.2562 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 284ms/step - loss: 0.2248 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 312ms/step - loss: 0.1738 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 292ms/step - loss: 0.1445 - accuracy: 1.0000


<TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
